In [1]:
import os

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline
import keras
import mne
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf
import random

from mne.channels import make_standard_montage
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model


In [2]:
data_dir = os.path.dirname("./data/")
data_files = os.listdir(data_dir)

In [3]:
def annotations_from_eGUI(raw, egui):
    codes = []
    starts = []

    current_state = None

    for i in range(len(egui)):
        if egui[i][0] != current_state:
            starts.append(i)
            current_state = egui[i][0]
            codes.append(str(egui[i][0]))

    starts.append(len(egui))
    codes = np.array(codes)
    sf = raw.info.get('sfreq')
    starts = np.array(starts) / sf
    durations = starts[1:] - starts[:-1]
    starts = starts[:-1]

    raw.set_annotations(mne.Annotations(onset=starts, duration=durations, description=codes))


def raw_from_mat(file):
    mat = loadmat(os.path.join(data_dir, file))

    sampling_freq = mat["o"][0][0][2][0][0]
    n_samples = mat["o"][0][0][3][0][0]
    ch_names = [element[0][0] for element in mat["o"][0][0][6]]

    df = pd.DataFrame(mat["o"][0][0][5], columns=ch_names)
    df = df.drop(columns=["X5"])
    df = df.T
    ch_names.remove("X5")

    ch_types = ['eeg'] * 21
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(df.to_numpy(), info)

    montage = make_standard_montage("standard_prefixed")
    raw.set_montage(montage)

    raw.load_data().set_eeg_reference(ref_channels='average')
    annotations_from_eGUI(raw, mat["o"][0][0][4])
    return raw


def filter_raw(raw):
    return raw.load_data().filter(0.1, 30, method="fir", phase="zero-double")

In [4]:
raw_NoMT = [raw_from_mat(file) for file in data_files if "NoMT" in file]
raw_FREEFORM = [raw_from_mat(file) for file in data_files if "FREEFORM" in file]

Creating RawArray with float64 data, n_channels=21, n_times=664400
    Range : 0 ... 664399 =      0.000 ...  3321.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=664600
    Range : 0 ... 664599 =      0.000 ...  3322.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=662400
    Range : 0 ... 662399 =      0.000 ...  3311.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=667600
    Range : 0 ... 667599 =      0.000 ...  3337.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 d

In [5]:
def get_epochs(raw):
    metadata_tmin, metadata_tmax = -1, 1

    all_events, all_event_id = mne.events_from_annotations(raw)
    metadata, events, event_id = mne.epochs.make_metadata(
        events=all_events,
        event_id=all_event_id,
        tmin=metadata_tmin,
        tmax=metadata_tmax,
        sfreq=raw.info["sfreq"],
    )
    return mne.Epochs(raw, events, event_id, reject_by_annotation=True)


In [6]:
epochs_NoMT = [get_epochs(file) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file) for file in raw_FREEFORM]

Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1931 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1919 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1925 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1935 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used

In [7]:
epochs_NoMT = [get_epochs(file) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file) for file in raw_FREEFORM]


Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1931 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1919 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1925 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1935 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used

In [8]:
epochs_NoMT[0].get_data().max()

Using data from preloaded Raw for 1931 events and 141 original time points ...
1 bad epochs dropped


865.3172938443672

In [9]:
epochs_FREEFORM[0].get_data().max()

Using data from preloaded Raw for 1481 events and 141 original time points ...
1 bad epochs dropped


158.92792102206735

In [10]:
epochs_data_NOMT = [file.get_data() for file in epochs_NoMT]
epochs_data_FREEFORM = [file.get_data() for file in epochs_FREEFORM]

Using data from preloaded Raw for 1930 events and 141 original time points ...
Using data from preloaded Raw for 1919 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1925 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1933 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1480 events and 141 original time points ...
Using data from preloaded Raw for 1383 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1409 events and 141 original time points ...
1 bad epochs dropped


In [11]:
stacked_NOMT = np.vstack(epochs_data_NOMT)
stacked_FREEFORM = np.vstack(epochs_data_FREEFORM)

In [12]:
stacked_NOMT.shape

(13506, 21, 141)

In [13]:
stacked_FREEFORM.shape

(4270, 21, 141)

In [14]:
np.random.shuffle(stacked_NOMT)
np.random.shuffle(stacked_FREEFORM)

In [15]:
X_nomt_train = stacked_NOMT[:12000]
X_nomt_test = stacked_NOMT[12000:]

In [16]:
X_free = stacked_FREEFORM

In [17]:
# make Freeform test set same length as NoMT
idy = random.sample(range(0, len(X_free)), X_nomt_test.shape[0])
X_free_test = X_free[idy]

In [18]:
X_free_test.shape

(1506, 21, 141)

In [19]:
def calc_accuracy(a, b, th):
    first = [1 if i < th else 0 for i in a]
    last = [1 if i > th else 0 for i in b]
    return sum(first + last) / len(first + last)

# Standard Autoencoder

In [20]:
layer = layers.Normalization()
layer1 = layers.Normalization()
layer.adapt(X_nomt_train.astype(float))
layer1.adapt(X_free_test.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(layer(X_nomt_train)))
print(np.max(layer1(X_free_test)))

[[[-1.42555168e+00  2.67063879e+00 -1.50269454e+00 ...  3.44254355e+00
    1.20120674e+01  1.12906388e+01]
  [-5.35384437e+00 -4.33765389e+00 -1.67098722e+00 ...  2.75425087e+00
    7.28377468e+00  4.69234611e+00]
  [ 6.78106852e-01  4.14297329e-01 -1.79036005e-01 ...  6.05620209e+00
    5.44572590e+00  1.94429733e+00]
  ...
  [ 7.57131243e-01  1.49332172e+00 -4.00116144e-02 ...  1.67522648e+00
    2.34475029e+00  3.13321719e-01]
  [ 2.09322880e+00  3.02941928e+00  2.28608595e+00 ...  4.54132404e+00
    2.32084785e+00  2.11941928e+00]
  [ 2.49054588e+00  2.40673635e+00  8.43403020e-01 ...  4.88864111e+00
    3.31816492e+00  4.41673635e+00]]

 [[-3.02198606e+00 -5.48674797e+00 -2.42103368e+00 ...  3.37718235e+01
    3.27508711e+01  2.54413473e+01]
  [ 9.12069686e+00  1.65934959e-01  3.05164925e+00 ...  5.55345064e+01
    5.12635540e+01  4.61340302e+01]
  [-2.23491289e+00 -3.13967480e+00 -3.35396051e+00 ...  8.32889663e+00
    8.55794425e+00  1.63842044e+00]
  ...
  [-3.48003484e+00 -6.3

In [21]:
latent_dim = 512
keras.backend.clear_session()


class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(512, activation='gelu'),
            layers.Dense(64, activation='gelu'),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(512, activation='gelu'),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(21 * 141, activation='linear'),
            layers.Reshape((21, 141))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


autoencoder = Autoencoder(latent_dim)

In [22]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())
autoencoder.fit(layer(X_nomt_train), layer(X_nomt_train),
                epochs=10,
                batch_size=64,
                shuffle=True,
                validation_data=(layer(X_nomt_test[:1000]), layer(X_nomt_test[:1000])))

Epoch 1/10
188/188 [==============================] - 5s 25ms/step - loss: 0.5668 - val_loss: 0.4270
Epoch 2/10
188/188 [==============================] - 4s 23ms/step - loss: 0.4495 - val_loss: 0.4038
Epoch 3/10
188/188 [==============================] - 5s 25ms/step - loss: 0.4169 - val_loss: 0.3569
Epoch 4/10
188/188 [==============================] - 5s 29ms/step - loss: 0.3681 - val_loss: 0.3828
Epoch 5/10
188/188 [==============================] - 6s 30ms/step - loss: 0.3749 - val_loss: 0.3430
Epoch 6/10
188/188 [==============================] - 6s 30ms/step - loss: 0.3713 - val_loss: 0.3313
Epoch 7/10
188/188 [==============================] - 6s 30ms/step - loss: 0.3706 - val_loss: 0.3583
Epoch 8/10
188/188 [==============================] - 6s 31ms/step - loss: 0.3762 - val_loss: 0.3333
Epoch 9/10
188/188 [==============================] - 6s 30ms/step - loss: 0.3590 - val_loss: 0.3254
Epoch 10/10
188/188 [==============================] - 6s 30ms/step - loss: 0.3460 - val_lo

In [23]:
err = []
err2 = []
for i in X_nomt_train:
    # need to expand here because the flatten layer assumes that the first dimension is the number of samples
    i = np.expand_dims(i, axis=0)
    encoded = autoencoder.encoder(layer(i)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(i-decoded)).mean())
    err.append((np.square(layer(i) - decoded)).mean())
print("###################")

for j in X_free_test:
    j = np.expand_dims(j, axis=0)
    encoded = autoencoder.encoder(layer1(j)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(layer1(j) - decoded)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())


###################
##############
0.34291136
0.61085016


In [24]:
calc_accuracy(err, err2, np.mean([np.array(err).mean(), np.array(err2).mean()]))


0.8157855767806901

# Convolutional Autoencoder

In [25]:
X_nomt_train = np.moveaxis(X_nomt_train, 1, 2)
X_nomt_test = np.moveaxis(X_nomt_test, 1, 2)
X_free_test = np.moveaxis(X_free_test, 1, 2)
X_free_test.shape

(1506, 141, 21)

In [26]:
# Adjust normalization layer to different shape
layer = layers.Normalization()
layer1 = layers.Normalization()
layer.adapt(X_nomt_train.astype(float))
layer1.adapt(X_free_test.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(layer(X_nomt_train)))
print(np.max(layer1(X_free_test)))

[[[-1.42555168e+00 -5.35384437e+00  6.78106852e-01 ...  7.57131243e-01
    2.09322880e+00  2.49054588e+00]
  [ 2.67063879e+00 -4.33765389e+00  4.14297329e-01 ...  1.49332172e+00
    3.02941928e+00  2.40673635e+00]
  [-1.50269454e+00 -1.67098722e+00 -1.79036005e-01 ... -4.00116144e-02
    2.28608595e+00  8.43403020e-01]
  ...
  [ 3.44254355e+00  2.75425087e+00  6.05620209e+00 ...  1.67522648e+00
    4.54132404e+00  4.88864111e+00]
  [ 1.20120674e+01  7.28377468e+00  5.44572590e+00 ...  2.34475029e+00
    2.32084785e+00  3.31816492e+00]
  [ 1.12906388e+01  4.69234611e+00  1.94429733e+00 ...  3.13321719e-01
    2.11941928e+00  4.41673635e+00]]

 [[-3.02198606e+00  9.12069686e+00 -2.23491289e+00 ... -3.48003484e+00
   -1.78320557e+00 -1.26686411e+00]
  [-5.48674797e+00  1.65934959e-01 -3.13967480e+00 ... -6.33479675e+00
   -2.82796748e+00  1.40837398e+00]
  [-2.42103368e+00  3.05164925e+00 -3.35396051e+00 ... -4.57908246e+00
   -3.32225319e+00  1.01408827e+00]
  ...
  [ 3.37718235e+01  5.5

In [27]:
latent_dim = 512
keras.backend.clear_session()


class ConvAutoencoder(Model):
    def __init__(self, latent_dim):
        super(ConvAutoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(X_nomt_train.shape[1], X_nomt_train.shape[2])),  # 141, 21
            layers.Conv1D(14, 3, activation=None, padding='same', strides=1),
            layers.LeakyReLU(),
            layers.Conv1D(7, 3, activation=None, padding='same', strides=1),
            layers.LeakyReLU(),
        ])

        self.decoder = tf.keras.Sequential([
            layers.Conv1DTranspose(7, kernel_size=3, strides=1, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.Conv1DTranspose(14, kernel_size=3, strides=1, activation=None, padding='same'),
            layers.LeakyReLU(),
            layers.Conv1D(X_nomt_train.shape[2], kernel_size=3, activation=None, padding='same'),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


conv_autoencoder = ConvAutoencoder(latent_dim)


In [28]:
conv_autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 141, 14)           896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 141, 14)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 141, 7)            301       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 141, 7)            0         
                                                                 
Total params: 1,197
Trainable params: 1,197
Non-trainable params: 0
_________________________________________________________________


In [29]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
conv_autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())

In [30]:
conv_autoencoder.fit(layer(X_nomt_train), layer(X_nomt_train),
                     epochs=10,
                     batch_size=64,
                     shuffle=True,
                     validation_data=(layer(X_nomt_test[:1000]), layer(X_nomt_test[:1000])))

Epoch 1/10
188/188 [==============================] - 3s 12ms/step - loss: 0.5577 - val_loss: 0.3651
Epoch 2/10
188/188 [==============================] - 2s 12ms/step - loss: 0.3592 - val_loss: 0.3046
Epoch 3/10
188/188 [==============================] - 2s 12ms/step - loss: 0.3076 - val_loss: 0.2680
Epoch 4/10
188/188 [==============================] - 2s 12ms/step - loss: 0.2728 - val_loss: 0.2406
Epoch 5/10
188/188 [==============================] - 2s 12ms/step - loss: 0.2475 - val_loss: 0.2221
Epoch 6/10
188/188 [==============================] - 2s 12ms/step - loss: 0.2302 - val_loss: 0.2110
Epoch 7/10
188/188 [==============================] - 2s 12ms/step - loss: 0.2198 - val_loss: 0.2036
Epoch 8/10
188/188 [==============================] - 2s 12ms/step - loss: 0.2122 - val_loss: 0.1976
Epoch 9/10
188/188 [==============================] - 2s 11ms/step - loss: 0.2066 - val_loss: 0.1918
Epoch 10/10
188/188 [==============================] - 2s 11ms/step - loss: 0.2016 - val_lo

In [31]:
err = []
err2 = []
for i in X_nomt_train:
    # need to expand here because the flatten layer assumes that the first dimension is the number of samples
    i = np.expand_dims(i, axis=0)
    encoded = conv_autoencoder.encoder(layer(i)).numpy()
    decoded = conv_autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(i-decoded)).mean())
    err.append((np.square(layer(i) - decoded)).mean())
print("###################")

for j in X_free_test:
    j = np.expand_dims(j, axis=0)
    encoded = conv_autoencoder.encoder(layer1(j)).numpy()
    decoded = conv_autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(layer1(j) - decoded)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())

###################
##############
0.19847809
0.26387638


In [32]:
calc_accuracy(err, err2, np.mean([np.array(err).mean(), np.array(err2).mean()]))

0.7764697171627425

In [33]:
calc_accuracy(err, err2, 1)

0.8728713164519473